In [1]:
from ultralytics import YOLO
import cv2
import numpy as np

In [2]:
model=YOLO('./YOLO weights/yolov8n.pt')

In [63]:
blank=np.zeros((480,640),dtype='uint8')
mask=cv2.rectangle(blank,(100,100),(440,480),(255,255,255),cv2.FILLED)
# cv2.imshow("mask",mask)
# cv2.waitKey(0)

In [64]:
import math

cap=cv2.VideoCapture("./images/people_walking_on_escalator.mp4")

left=0
right=0

while True:
    ret,img=cap.read()
    if ret:
        img=cv2.resize(img,(640,480))
        img_crop=cv2.bitwise_and(img,img,mask=mask)
        results=model(img_crop,stream=True,verbose=False)
        
        for r in results:
            boxes=r.boxes
            for box in boxes:
                x1,y1,x2,y2=box.xyxy[0]
                x1,y1,x2,y2=int(x1),int(y1),int(x2),int(y2)
                
                classs=int(box.cls[0])
                conf=math.ceil(box.conf[0]*100)/100
                
                if classs==0:
                    if x1<=300:
                        if (y2>=238 and y2<=241):
                            left+=1
                        else:
                            continue
                    elif x2>=360:
                        if (y2>=239 and y2<=242):
                            right+=1
                    else:
                        continue
                    cv2.rectangle(img,(x1,y1),(x2,y2),(0,0,255),2)
        
        
        
        #left escalator
        cv2.line(img,(180,240),(250,240),(255,0,0),2) #horizontal
        cv2.line(img,(250,0),(250,480),(255,0,0),2) #vertical
        
        #right escalator
        cv2.line(img,(360,240),(420,240),(255,0,0),2) #horizontal
        cv2.line(img,(360,0),(360,480),(255,0,0),2) #vertical
        
        cv2.putText(img,f'left: {left}',(180,440),cv2.FONT_HERSHEY_COMPLEX,1,(0,200,200),2)
        cv2.putText(img,f'right: {right}',(360,440),cv2.FONT_HERSHEY_COMPLEX,1,(0,200,200),2)
        
        cv2.imshow("img",img)
        cv2.imshow("mask",img_crop)
        if cv2.waitKey(1) & 0xFF==ord('q'):
            break
    else:
        break
        
cap.release()
cv2.destroyAllWindows()
    
print("left people: ",left,"\nright people: ",right)

left people:  2 
right people:  5
